In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gemma/keras/gemma_2b_en/2/config.json
/kaggle/input/gemma/keras/gemma_2b_en/2/tokenizer.json
/kaggle/input/gemma/keras/gemma_2b_en/2/metadata.json
/kaggle/input/gemma/keras/gemma_2b_en/2/model.weights.h5
/kaggle/input/gemma/keras/gemma_2b_en/2/assets/tokenizer/vocabulary.spm
/kaggle/input/qa-dataset-foodie-finder/qa_pairs_df.csv


In [2]:
%%time
import os

os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

import keras
import keras_nlp

CPU times: user 8.97 s, sys: 1.25 s, total: 10.2 s
Wall time: 14.8 s


In [3]:
%%time
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


CPU times: user 1min 9s, sys: 48.9 s, total: 1min 58s
Wall time: 1min 28s


In [4]:
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

In [5]:
#!kaggle datasets download -d kjeevan/qa-pairs-foodiefinder

#!kaggle datasets download -d kjeevan/qa-pairs-foodie-finder

In [6]:
# #Download the model into your working directory 
# import kagglehub

# # Download latest version
# path = kagglehub.model_download("google/gemma/transformers/2b-it")

# print("Path to model files:", path)

Working only a subset of summarized data. Due to limitations, I took only the restaurants that have more than 28 reviews and further took only first 5 reviews for these restaurants. Summarized the reviews by separating into 5 dataframes. Each of these dataframes were provided to a kaggle notebook. Next restaurant and menu data was combined and data preprocessing on other columns were also performed. The final dataset contains approx 100 total restaurants and 500 reviews. Will be using this to experiment fine tuning.

In [7]:
import pandas as pd
qa_pairs = pd.read_csv("/kaggle/input/qa-dataset-foodie-finder/qa_pairs_df.csv")
qa_pairs.head()

,question,answer
0,What type of food does the restaurant Bistro S...,"Bistro, Italian, Bar"
1,What is the amount/price normally spent for di...,"JPY 5,000 - JPY 5,999"
2,What are some of the dishes served by Bistro S...,"Sake (Nihonshu),Shochu (Japanese spirits)"
3,What is the average rating of Bistro Sakaba RE...,3.5
4,What are the overall views of the customers on...,The reviewer visited the restaurant with a gro...


In [8]:
qa_pairs.shape

(1121, 2)

In [9]:
qa_pairs.dropna(inplace=True)
qa_pairs.shape

(1121, 2)

In [10]:
template = "Instruction:\n{question}\n\nResponse:\n{answer}"

In [11]:
# qa_pairs_test = qa_pairs_test = qa_pairs[
#     (qa_pairs['answer'] == 'Shinjuku-ku') |  # First condition
#     (qa_pairs['question'].str.contains('Shinjuku Sushi Yokota', case=False, na=False)) |  # Modified condition
#     (qa_pairs['question'].str.contains('Tempura Azabu Yokota Shinjuku Bettei', case=False, na=False)) 
# ]

# print(qa_pairs_test.shape)
# qa_pairs_test.head()

In [12]:
# %%time

# # answer after fine tuning
# prompt = template.format(
#     question="What food type is popular in Japan?",
#     answer="",
# )
# print(gemma_lm.generate(prompt, max_length=500))

In [13]:
# %%time

# # answer after fine tuning
# prompt = template.format(
#     question="Suggest some restaurants in Tokyo.",
#     answer="",
# )
# print(gemma_lm.generate(prompt, max_length=500))

In [14]:
# %%time

# prompt = template.format(
#     question="List 3 popular restaurants in Kyoto and what people think about these restaurants.",
#     answer="",
# )
# print(gemma_lm.generate(prompt, max_length=500))

In [15]:
# %%time
# # Import necessary libraries (modify paths according to your setup)
# from keras_nlp.models import GemmaTokenizer

# # Load the tokenizer specific to your Gemma model (assuming 'gemma_2b_en')
# tokenizer = GemmaTokenizer.from_preset('gemma_2b_en')

# # Sample function to tokenize and compute average length
# def get_average_token_length(qa_dataset):
#     token_lengths = []
#     for index, row in qa_dataset.iterrows():
#         # Combine question and answer to tokenize them together
#         text = row['question'] + " " + row['answer']
        
#         # Tokenize using Gemma's tokenizer
#         tokens = tokenizer.tokenize(text)
        
#         # Append the token length of the current row
#         token_lengths.append(len(tokens))
    
#     # Return the average token length
#     return sum(token_lengths) / len(token_lengths) if len(token_lengths) > 0 else 0


# # Calculate the average token length
# average_length = get_average_token_length(qa_pairs)
# print(f"Average token length: {average_length}")


In [16]:
data = []
for index, row in qa_pairs.iterrows():
    # Use the template to format the string and append it to the data list
    formatted_string = template.format(question=row['question'], answer=row['answer'])
    data.append(formatted_string)

In [17]:
# Enable LoRA for the model and set the LoRA rank to 4.
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,507,536,384 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,507,536,384 (9.34 GB)

 Trainable params: 1,363,968 (5.20 MB)

 Non-trainable params: 2,506,172,416 (9.34 GB)

In [18]:
%%time

# configure and run a training session

gemma_lm.preprocessor.sequence_length = 80


optimizer = keras.optimizers.AdamW(
    learning_rate=4e-5, # 2e-4
    weight_decay=0.01,
)


optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])


gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

CPU times: user 5.25 ms, sys: 150 µs, total: 5.4 ms
Wall time: 4.92 ms


In [19]:
%%time
gemma_lm.fit(data, epochs=2, batch_size=5)

Epoch 1/2
225/225 ━━━━━━━━━━━━━━━━━━━━ 12653s 56s/step - loss: 2.4048 - sparse_categorical_accuracy: 0.4063
Epoch 2/2
225/225 ━━━━━━━━━━━━━━━━━━━━ 12648s 56s/step - loss: 1.6685 - sparse_categorical_accuracy: 0.5707
CPU times: user 1d 23min 42s, sys: 44min 8s, total: 1d 1h 7min 51s
Wall time: 7h 1min 42s


In [20]:
%%time
gemma_lm.save('gemma_lm_model.h5') 

CPU times: user 792 ms, sys: 32.3 s, total: 33.1 s
Wall time: 42.2 s


In [21]:
# %%time
# # Save only the model weights
# gemma_lm.save_weights('gemma_lm_weights.weights.h5')

In [22]:
%%time

# answer after fine tuning
prompt = template.format(
    question="What food type is popular in Japan?",
    answer="",
)
print(gemma_lm.generate(prompt, max_length=500))

Instruction:
What food type is popular in Japan?

Response:
The most popular food in Japan is sushi. It is a type of Japanese cuisine that consists of raw fish, usually
CPU times: user 2min 46s, sys: 2.76 s, total: 2min 48s
Wall time: 1min 14s


In [23]:
%%time

# answer after fine tuning
prompt = template.format(
    question="What food category people usually eat in Hokkaido?",
    answer="",
)
print(gemma_lm.generate(prompt, max_length=500))

Instruction:
What food category people usually eat in Hokkaido?

Response:
Hokkaido is known for its fresh seafood, and the region is home to many famous restaurants serving up delicious dishes made with local ingredients. Some of the most popular dishes in Hokkaido include grilled seafood, such as grilled scallops and grilled salmon, as well as dishes made with Hokkaido's famous dairy products, such as Hokkaido milk ice cream and Hokkaido cheese.
CPU times: user 4min 28s, sys: 1.74 s, total: 4min 29s
Wall time: 2min 19s


In [24]:
%%time

# answer after fine tuning
prompt = template.format(
    question="What are some restaurants in Tokyo and what do people think about these restaurants?", # Suggest some restaurants in Tokyo.
    answer="",
)
print(gemma_lm.generate(prompt, max_length=500))

Instruction:
What are some restaurants in Tokyo and what do people think about these restaurants?

Response:
The restaurant is located in the heart of Tokyo, and the atmosphere is very lively. The menu is extensive, and there is something for everyone. The food is delicious, and the service is excellent.
CPU times: user 3min 49s, sys: 2.27 s, total: 3min 51s
Wall time: 1min 43s


In [25]:
%%time

# answer after fine tuning
prompt = template.format(
    question="What are some restaurants in Kyoto?", # List 3 popular restaurants in Kyoto and what people think about these restaurants.
    answer="",
)
print(gemma_lm.generate(prompt, max_length=500))

Instruction:
What are some restaurants in Kyoto?

Response:
The Kyoto Prefecture is home to a variety of delicious and unique dishes. Some of the most popular dishes in the region include the Kyoto-style sushi, which is known for its fresh and flavorful ingredients. The Kyoto-style sushi is typically served with a variety of sauces and toppings, such as wasabi and pickled ginger. Another popular dish in the region is the Kyoto-style ramen, which is a type of noodle soup that is typically served with a variety of toppings, such as pork, chicken, and vegetables. The Kyoto-style ramen is known for its rich and flavorful broth, which is often made with a variety of spices and herbs.
CPU times: user 7min 11s, sys: 2.04 s, total: 7min 13s
Wall time: 3min 59s
